<a href="https://colab.research.google.com/github/smara97/FakeNews/blob/master/Fake_New_TF_IDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import  drive
import re
import string 
import torch
import torch.nn as nn
import nltk
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from string import punctuation
from collections import Counter
import numpy as np
import pandas as pd
from torch.utils.data import TensorDataset, DataLoader
import matplotlib.pyplot as plt
from gensim.models import Doc2Vec
from gensim.models.doc2vec import LabeledSentence
from gensim import utils
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.feature_extraction.text import TfidfVectorizer
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
trainingset=pd.read_csv("/content/drive/My Drive/Fake news competitive/finalstraingv1.csv")
trainingset=trainingset.drop([trainingset.columns[0]],axis=1)
trainingset=trainingset.dropna()


In [0]:
def clean(text):
  text=text.lower()
  stp=set(stopwords.words("english"))
  placesp = re.compile('[/(){}\[\]\|@,;]')
  removech= re.compile('[^0-9a-z #+_]')
  st=PorterStemmer()
  text=re.sub(placesp,' ',text)
  text=re.sub(removech,' ',text)
  text=text.split()
  text=[w for w in text if not w in stp]
  text=[st.stem(w) for w in text]
  text=" ".join(text)
  text = text.translate(str.maketrans("", "", string.punctuation))
  return text

In [0]:
trainingset['total']=trainingset['author']+' , '+trainingset['title']+' , '+trainingset['text']
labels=trainingset['label']
lbls=labels
trainingset=trainingset.drop(['label'],axis=1)

In [0]:
for i in range(trainingset.shape[0]):
  try:
    trainingset.loc[i,'total']=clean(trainingset.loc[i,'total'])
  except:
    continue


In [0]:
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1,5), max_df=1.0, min_df=0,token_pattern='(\S+)')
trainX= tfidf_vectorizer.fit_transform(trainingset.total)

In [0]:
tfidf_reversed_vocab = {i:word for word,i in tfidf_vectorizer.vocabulary_.items()}
trainxx=trainX[:int(trainX.shape[0]*0.85)]
testx=trainX[int(trainX.shape[0]*0.85)+1:]
lablt=lbls[:int(trainX.shape[0]*0.85)]
lablts=lbls[int(trainX.shape[0]*0.85)+1:]


In [9]:
model = OneVsRestClassifier(LogisticRegression(penalty='l2', C=1.0))
model.fit(trainxx, lablt)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


OneVsRestClassifier(estimator=LogisticRegression(C=1.0, class_weight=None,
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=100,
                                                 multi_class='warn',
                                                 n_jobs=None, penalty='l2',
                                                 random_state=None,
                                                 solver='warn', tol=0.0001,
                                                 verbose=0, warm_start=False),
                    n_jobs=None)

In [0]:
y_pred=model.predict(testx)

In [11]:
num_co=(y_pred==lablts).sum()
print(num_co/len(lablts))

0.7892167739072554


In [29]:
author,title,text="Smara","Assiut","Assiut is a goverment in Egypt"
text=clean(author+' , '+' , '+title+' , '+text)
tx=[]
tx.append(text)
tx=tfidf_vectorizer.transform(tx)
ypred=model.predict(tx)
if ypred == 0:
  print("REAL :)")
else:
  print("FAKE :(")
print(model.decision_function(tx))

REAL :)
[-0.05555335]
